# Visit individual Calexp_Background from Selected visits

- author Sylvie Dagoret-Campagne
- creation date 2024-05-04
- last update 2024-05-06
- affiliation : IJCLab
- kernel : **w_2024_16**



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collectionn = collection2
#collections = [collection1,collection2,collection3]
collections = [collection2]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"
NDET = 9
suptitle=  collectionStr

tract = 9880 # mostly for light-curves
detector = 0
band = 'g'

suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)


In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
#file_selected_visits = os.path.join("../Visits",fn_ccdVisit_tracts_patches)  
#df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)
#cut = (df_myselectedvisits.band == band) &  (df_myselectedvisits.tractID == tract) 
#df = df_myselectedvisits[cut]

In [ ]:
if 1:
    # get the csv file produced by stat_on_visits_LSSTAuxtel.ipynb in ../Visits
    file_selected_visits = os.path.join("../Visits",fn_ccdVisit_tracts_patches)
    
  
    df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)

    # select the band
    cut = (df_myselectedvisits.band == band) &  (df_myselectedvisits.tractID == tract) & (df_myselectedvisits.detector == detector)
    df_myselectedvisits = df_myselectedvisits[cut]

    #move the visitid as a column not to loose it during the merge 
    df_myselectedvisits.reset_index(inplace=True) 


In [ ]:
index = 10
visitId = df_myselectedvisits.iloc[index]['visitId']

In [ ]:
datasetType = 'calexpBackground'
dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)

for i, ref in enumerate(datasetRefs):
    print(ref.dataId)
    print("band:", ref.dataId['band'])
    band = ref.dataId['band']

## One Calexp Background

In [ ]:
%matplotlib widget

bkgd = butler.get('calexpBackground', **dataId,collections=collections)
img = bkgd.getImage()
#wcs = img.getWcs()
#bbox = img.getBBox()
#corners_ra, corners_dec = get_corners_radec(wcs, bbox)
#extent = [np.min(corners_ra),np.max(corners_ra),np.min(corners_dec),np.max(corners_dec)]


# Display the image with a suitable scaling
data_flat = img.array.flatten()
med = np.median(data_flat)
sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
vmin = med - 3 * sigMad
vmax = med + 3 * sigMad

fig,axs = plt.subplots(1,1,figsize=(8,6))
ax = axs
im=ax.imshow(img.array, origin="lower",vmin=vmin, vmax=vmax, cmap="Greys")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"Calexp : visit = {visitId}, band = {band}"
ax.set_title(the_title)
ax.grid()
ax.set_aspect('equal')
plt.colorbar(im, cax=cax,ax=ax)


plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Many Background

In [ ]:
df_myselectedvisits.tail(100)

In [ ]:
NROWS = 10
NCOLS = 4
NIMGS = NROWS*NCOLS
index0 = 0

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*4,NROWS*3))

for index,ax in enumerate(axs.flatten()):
    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    bkgd = butler.get('calexpBackground', **dataId,collections=collections)
    img = bkgd.getImage()

    data_flat = img.array.flatten()
    med = np.median(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin = med - 3 * sigMad
    vmax = med + 3 * sigMad

    im=ax.imshow(img.array, origin="lower",vmin=vmin, vmax=vmax, cmap="grey")
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    the_title = f"{visitId},({band},{detector})"
    ax.set_title(the_title)
    ax.grid()
    ax.set_aspect('equal')
    plt.colorbar(im, cax=cax,ax=ax)

plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Background flux (nJ)

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*3,NROWS*3))

for index,ax in enumerate(axs.flatten()):
    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    bkgd = butler.get('calexpBackground', **dataId,collections=collections)
    img = bkgd.getImage()

    data_flat = img.array.flatten()
    mu = np.mean(data_flat)
    med = np.median(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin = med - 3 * sigMad
    vmax = med + 3 * sigMad

    
    textstr = '\n'.join((
    r'$\mu=%.2f$ nJ/pix' % (mu, ),
    r'$\mathrm{med}=%.2f$ nJ/pix' % (med, ),
    r'$\mathrm{med}=%.2f$ mag/arcsec2' % (convert_fluxtomag(med/pixel_scale**2), ),
    r'$\sigma=%.2f$ nJ/pix' % (sigMad, ),
    r'$\sigma=%.1f$ mag/arcsec2' % (convert_fluxtomag(sigMad/pixel_scale**2), )))

    histdata = ax.hist(data_flat,bins=50,range=(vmin,vmax),histtype="step",color="b",lw=2)
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    
    the_title = f"{visitId},{band}"
    ax.set_title(the_title)
    ax.set_xlabel("Background flux (nJ)")
    ax.grid()
    ax.set_ylim(0.,histdata[0].max()*2.)
  
plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Background flux (nJ) in not masked pixels

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*3,NROWS*3))

for index,ax in enumerate(axs.flatten()):
    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)
    mask = calexp.mask.array
    the_mask = np.where(mask==0,1,0)
    bkgd = butler.get('calexpBackground', **dataId,collections=collections)
    img = bkgd.getImage().array* the_mask

    data_flat = img.flatten()
    data_flat_nozero = data_flat[~(data_flat==0)]
    mu = np.mean(data_flat_nozero )
    med = np.median(data_flat_nozero )
    sigMad_nozero  = 1.4826 * np.median(np.fabs(data_flat_nozero  - med))
    vmin = med - 3 * sigMad_nozero 
    vmax = med + 3 * sigMad_nozero 

    
    textstr = '\n'.join((
    r'$\mu=%.2f$ nJ/pix' % (mu, ),
    r'$\mathrm{med}=%.2f$ nJ/pix' % (med, ),
    r'$\mathrm{med}=%.2f$ mag/arcsec2' % (convert_fluxtomag(med/pixel_scale**2), ),
    r'$\sigma=%.2f$ nJ/pix' % (sigMad_nozero , ),
    r'$\sigma=%.1f$ mag/arcsec2' % (convert_fluxtomag(sigMad_nozero/pixel_scale**2 ), )))

    histdata = ax.hist(data_flat,bins=50,range=(vmin,vmax),histtype="step",color="b",lw=2)
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    
    the_title = f"{visitId},{band}"
    ax.set_title(the_title)
    ax.set_xlabel("Background flux (nJ/pixel)")
    ax.grid()
    ax.set_ylim(0.,histdata[0].max()*2.)
  
plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Background in Mag

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*3,NROWS*3))

for index,ax in enumerate(axs.flatten()):
    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    bkgd = butler.get('calexpBackground', **dataId,collections=collections)
    img = bkgd.getImage()

    data_flat = img.array.flatten()
    med = np.median(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin = med - 3 * sigMad
    vmax = med + 3 * sigMad

    data_flat_mag = -2.5*np.log10((data_flat/med))*1000
    mu_mag = np.mean(data_flat_mag)
    med_mag = np.median(data_flat_mag)
    sigMad_mag = 1.4826 * np.median(np.fabs(data_flat_mag - med_mag))
    vmin = med_mag - 3 * sigMad_mag
    vmax = med_mag + 3 * sigMad_mag
    textstr = '\n'.join((
    r'$\mu=%.2f$ mmag' % (mu_mag, ),
    r'$\mathrm{med}=%.2f$ mmag' % (med_mag, ),
    r'$\sigma=%.2f$ mmag' % (sigMad_mag , )))

    histdata = ax.hist(data_flat_mag,bins=50,range=(vmin,vmax),histtype="step",color="b",lw=2)
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    
    the_title = f"{visitId},({band},{detector})"
    ax.set_title(the_title)
    ax.set_xlabel("rel Mag (mmag)")
    ax.grid()
    ax.set_ylim(0.,histdata[0].max()*1.4)
  
plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)